In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import random
import pickle
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
hm_lines = 100000
import pickle

In [ ]:
def create_lexicon(pos,neg):
    # lexicon will be collection of all words
    lexicon = []
    # with opens kee
    with open(pos,'r') as f:
        contents = f.readlines()
    for l in contents[:hm_lines]:
        all_words = word_tokenize(l)
        lexicon += list(all_words)

    with open(neg,'r') as f:
        contents = f.readlines()
    for l in contents[:hm_lines]:
        all_words = word_tokenize(l)
        lexicon += list(all_words)
    
    # storing all words by removing the 
    lexicon = [lemmatizer.lemmatize(i) for i in lexicon if i not in stopwords.words('english')]
    # its like freq count in nltk stores count of each word with 
    w_counts = Counter(lexicon)
    l2 = []
    for w in w_counts:
        #print(w_counts[w])
        if w_counts[w] > 50:
            l2.append(w)
    print(len(l2))
    return l2

def sample_handling(sample,lexicon,classification):

    featureset = []
    with open(sample,'r') as f:
        contents = f.readlines()
    for l in contents:
        current_words = word_tokenize(l.lower())
        current_words = [lemmatizer.lemmatize(i) for i in current_words]
        features = np.zeros(len(lexicon))
        for word in current_words:
            if word.lower() in lexicon:
                # gives index at which the particular word is stored in lexicon
                index_value = lexicon.index(word.lower())
                features[index_value] += 1

        features = list(features)
        featureset.append([features,classification])

    return featureset

def create_feature_sets_and_labels(pos,neg,test_size = 0.1):
    
    lexicon = create_lexicon(pos,neg)
    features = []
    features += sample_handling('Data/positive.txt',lexicon,[1,0])
    features += sample_handling('Data/negetive.txt',lexicon,[0,1])
    random.shuffle(features)
    features = np.array(features)

    testing_size = int(test_size*len(features))

    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])
    
    # my added code
    
#     with open('Data/sentiment_set.pickle','wb') as f:
#         pickle.dump(features,f)

    return train_x,train_y,test_x,test_y

if __name__ == '__main__':
    train_x,train_y,test_x,test_y = create_feature_sets_and_labels('Data/positive.txt','Data/negetive.txt')
#     create_feature_sets_and_labels('Data/positive.txt','Data/negetive.txt')
#     Saving the data to pickle file:
    with open('Data/sentiment_set.pickle','wb') as f:
        pickle.dump([train_x,train_y,test_x,test_y],f)

In [ ]:
# file = open('Data/sentiment_set.pickle','rb')
# train_X,train_Y,test_X,test_Y = pickle.load(file) 

In [ ]:
# train_X